In [87]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
import os

import mlflow
import segmentation_models_pytorch as smp
from torchvision.transforms import ToPILImage

from cosas.tracking import get_experiment
from cosas.paths import DATA_DIR
from cosas.data_model import COSASData, Scanncers
from cosas.datasets import Patchdataset
from cosas.misc import set_seed, train_val_split

In [89]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

DEVICE = "cuda"
# experiment = get_experiment()
set_seed(42)  # TODO

cosas_data = COSASData(os.path.join(DATA_DIR, "task2"))
cosas_data.load()

(train_images, train_masks), (val_images, val_masks), (test_images, test_masks) = (
    train_val_split(cosas_data, train_val_test=(0.6, 0.2, 0.2))
)

model = smp.FPN(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    classes=1,
    activation=None,
).to(DEVICE)


Wholedataset

In [90]:
from matplotlib import pyplot as plt

from cosas.datasets import WholeSizeDataset
wholesize_ds = WholeSizeDataset(train_images, train_masks)
image, mask = wholesize_ds[0]

patches = wholesize_ds._tesellation(image)
batch = patches.shape[0]

fig, ax = plt.subplots(figsize=(6, 6), nrows=6, ncols=7)
for n, i in enumerate(patches):
    ax.flat[n].imshow(ToPILImage()(i))
    ax.flat[n].axis("off")

plt.tight_layout()
plt.show()

ValueError: too many values to unpack (expected 3)

In [91]:
import torch
from torch.utils.data import DataLoader
from cosas.metrics import calculate_metrics, Metrics, AverageMeter
from cosas.trainer import BinaryClassifierTrainer
from cosas.tracking import TRACKING_URI, get_experiment

train_transform = A.Compose(
        [
            A.RandomCrop(height=224, width=224, p=1),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            # TODO: RandomRotation90 추가
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ]
    )

train_dataset = Patchdataset(train_images, train_masks, train_transform)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
)

test_transform = A.Compose(
    [
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
val_dataset = WholeSizeDataset(val_images, val_masks, test_transform)
num_epochs = 2
n_steps = len(train_dataloader)
trainer = BinaryClassifierTrainer(
    model, 
    torch.nn.functional.binary_cross_entropy_with_logits, 
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
)

mlflow.set_tracking_uri(TRACKING_URI)
experiment = get_experiment("cosas")

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="test_run") as run:
    trainer.train(
        train_dataloader,
        val_dataset,
        1,
        7,
    )
    mlflow.pytorch.log_model(trainer.model, "model")
    

    test_loss, test_metrics = trainer.test(WholeSizeDataset(test_images, test_masks, test_transform), phase="test")
    mlflow.log_metric("test_loss", test_loss.avg)
    mlflow.log_metrics(test_metrics.to_dict(prefix="test_"))    

 |################################| train | EPOCH 0: [17/18] | eta:1 | total_loss: 1.1124 | f1: 0.6079|acc: 0.6728|sen: 0.6811|spec: 0.6903|auroc: 0.7645|prauc: 0.7407|iou: 0.4641|dice: 0.6079
 |####                            | val | EPOCH 0: [2/24] | eta:76 | total_loss: 0.8181 | f1: 0.7319|acc: 0.7501|sen: 0.6442|spec: 0.8724|auroc: 0.8660|prauc: 0.8887|iou: 0.5912|dice: 0.7319/home/heon/anaconda3/envs/cosas/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/heon/anaconda3/envs/cosas/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
 |########                        | val | EPOCH 0: [5/24] | eta:53 | total_loss: 0.9755 | f1: 